In [ ]:
### 로지스틱회귀분석

In [11]:
import pandas as pd          
from sklearn import model_selection
from sklearn import metrics
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#타이타닉 데이터 준비 
file_path= 'C:/data/titanic.csv'

#read csv()함수로 데이터프레임 변환 
df = pd.read_csv(file_path)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [12]:
## 데이터 전처리
#불필요 컬럼 삭제 (x)
del df['PassengerId']
del df['Name']
del df['Ticket']
del df['Cabin']

#결측치 처리 (x)
df.dropna(thresh=int(len(df) * 0.5), axis=1) #결측치 제거
df['Embarked'] = df['Embarked'].fillna('S') # 최다빈도 ‘S’로 대체
df['Age']=df['Age'].fillna(df['Age'].mean()) # 나이 평균값으로 대체


df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S
887,1,1,female,19.000000,0,0,30.0000,S
888,0,3,female,29.699118,1,2,23.4500,S
889,1,1,male,26.000000,0,0,30.0000,C


In [13]:
#범주형 컬럼 처리 (x) 
df['Sex']=df['Sex'].astype('category') 
df['Pclass']=df['Pclass'].astype('category') 
df['Embarked']=df['Embarked'].astype('category')
df = pd.get_dummies(df)
df

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,22.000000,1,0,7.2500,0,0,1,0,1,0,0,1
1,1,38.000000,1,0,71.2833,1,0,0,1,0,1,0,0
2,1,26.000000,0,0,7.9250,0,0,1,1,0,0,0,1
3,1,35.000000,1,0,53.1000,1,0,0,1,0,0,0,1
4,0,35.000000,0,0,8.0500,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.000000,0,0,13.0000,0,1,0,0,1,0,0,1
887,1,19.000000,0,0,30.0000,1,0,0,1,0,0,0,1
888,0,29.699118,1,2,23.4500,0,0,1,1,0,0,0,1
889,1,26.000000,0,0,30.0000,1,0,0,0,1,1,0,0


In [16]:
#독립변수와 종속변수 구분 (x, y)
x_data=df.iloc[:,1:]
y_data=df.iloc[:,0]
x_data = x_data.values
y_data = y_data.values
print(x_data)
print(y_data)


[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0
 0 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0
 0 1 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0
 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 0 1 0 0 1
 1 1 1 1 1 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0
 0 0 1 1 0 1 0 0 1 0 0 0 

In [18]:
## 머신러닝 
#데이터분할(7:3)
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_data, y_data, test_size=0.3)

#로지스틱 모델링 
estimator = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)
estimator.fit(x_train, y_train)

# train 평가 
y_predict = estimator.predict(x_train)
score = metrics.accuracy_score(y_train, y_predict) #classification
print('train score: ', score)

# test 평가 
y_predict = estimator.predict(x_test)
score = metrics.accuracy_score(y_test, y_predict)
print('test score: ', score)



train score:  0.7993579454253612
test score:  0.7947761194029851


In [22]:
# 모델 적용 
print(x_test[:2])
y_predict = estimator.predict(x_test[:2])
print(y_predict) 
for y1, y2 in zip(y_test, y_predict):
    print(y1, y2, y1==y2)


1 1 True
1 0 False


In [23]:
###의사결정나무

In [24]:
import pandas as pd          
from sklearn import model_selection
from sklearn import metrics
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#타이타닉 데이터 준비 
file_path= 'C:/data/titanic.csv'

#read csv()함수로 데이터프레임 변환 
df = pd.read_csv(file_path)
df

## 데이터 전처리
#불필요 컬럼 삭제 (x)
del df['PassengerId']
del df['Name']
del df['Ticket']
del df['Cabin']

#결측치 처리 (x)
df.dropna(thresh=int(len(df) * 0.5), axis=1) #결측치 제거
df['Embarked'] = df['Embarked'].fillna('S') # 최다빈도 ‘S’로 대체
df['Age']=df['Age'].fillna(df['Age'].mean()) # 나이 평균값으로 대체

#범주형 컬럼 처리 (x) 
df['Sex']=df['Sex'].astype('category') 
df['Pclass']=df['Pclass'].astype('category') 
df['Embarked']=df['Embarked'].astype('category')
df = pd.get_dummies(df)

#독립변수와 종속변수 구분 (x, y)
x_data=df.iloc[:,1:]
y_data=df.iloc[:,0]
x_data = x_data.values
y_data = y_data.values
print(x_data)
print(y_data)


[[22.          1.          0.         ...  0.          0.
   1.        ]
 [38.          1.          0.         ...  1.          0.
   0.        ]
 [26.          0.          0.         ...  0.          0.
   1.        ]
 ...
 [29.69911765  1.          2.         ...  0.          0.
   1.        ]
 [26.          0.          0.         ...  1.          0.
   0.        ]
 [32.          0.          0.         ...  0.          1.
   0.        ]]
[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 

In [26]:
## 머신러닝 
#데이터분할(7:3)
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_data, y_data, test_size=0.3)


#의사결정나무 모델링 
estimator = DecisionTreeClassifier(criterion='gini', max_depth=None, max_leaf_nodes=None, min_samples_split=2, min_samples_leaf=1, max_features=None)
estimator.fit(x_train, y_train)

# train 평가 
y_predict = estimator.predict(x_train) 
score = metrics.accuracy_score(y_train, y_predict)
print('train score: ', score)

y_predict = estimator.predict(x_test) 
score = metrics.accuracy_score(y_test, y_predict)
print('test score: ', score)



train score:  0.9823434991974318
test score:  0.7798507462686567


In [27]:
# 모델 검증 
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))


[[133  35]
 [ 24  76]]
              precision    recall  f1-score   support

           0       0.85      0.79      0.82       168
           1       0.68      0.76      0.72       100

    accuracy                           0.78       268
   macro avg       0.77      0.78      0.77       268
weighted avg       0.79      0.78      0.78       268



In [28]:
# 중요도 
estimator.fit(x_train, y_train)
print(estimator.feature_importances_)

d = {'attribute': df.iloc[:,1:].columns,
    'importance': estimator.feature_importances_ }
df_importance = pd.DataFrame(d)
df_importance.sort_values('importance', ascending=False)
print(df_importance)


[0.24428943 0.05361749 0.00901061 0.23944243 0.0345631  0.00566585
 0.06681305 0.         0.32160904 0.01040478 0.00097152 0.01361271]
     attribute  importance
0          Age    0.244289
1        SibSp    0.053617
2        Parch    0.009011
3         Fare    0.239442
4     Pclass_1    0.034563
5     Pclass_2    0.005666
6     Pclass_3    0.066813
7   Sex_female    0.000000
8     Sex_male    0.321609
9   Embarked_C    0.010405
10  Embarked_Q    0.000972
11  Embarked_S    0.013613


In [48]:
# 다운로드 받은 경로 입력 ('C:/Program Files (x86)/Graphviz/bin')
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz/bin'

#
from sklearn.tree import export_graphviz
export_graphviz(estimator, out_file="./tree.dot")
#export_graphviz(estimator, out_file="./tree.dot", feature_names=["hours", "attendance"]), class_names=["fail", "pass"],)


In [49]:
import graphviz
with open("./tree.dot") as f:
    dot_graph = f.read()
dot = graphviz.Source(dot_graph)
dot.format = 'png'
dot.render(filename='tree', directory='./', cleanup=True)
#
#dot


'.\\tree.png'

In [50]:

import webbrowser
webbrowser.open('tree.png')

True